In [2]:
import pandas as pd
import time
import pprint

# Leitura dos Dados

In [5]:
# Dataset read
df = pd.read_csv('data/unpacked/ca-2019-2022.csv', delimiter=';')
df.head(5)

C:\Users\bruno\AppData\Local\Temp\ipykernel_14852\147497070.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/unpacked/ca-2019-2022.csv', delimiter=';')


,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,GASOLINA,01/07/2019,"4,259",NaN,R$ / litro,BRANCA
1,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,ETANOL,01/07/2019,"4,099",NaN,R$ / litro,BRANCA
2,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,GNV,01/07/2019,"3,449",NaN,R$ / m³,BRANCA
3,NE,BA,ITABUNA,LOPES LEMOS COMERCIO DE COMBUSTIVEIS LTDA,00.231.792/0001-05,RODOVIA BR 101,SN,KM 503 5,MANOEL LEAO,45601-402,GASOLINA,01/07/2019,"4,69",NaN,R$ / litro,BRANCA
4,NE,BA,ITABUNA,LOPES LEMOS COMERCIO DE COMBUSTIVEIS LTDA,00.231.792/0001-05,RODOVIA BR 101,SN,KM 503 5,MANOEL LEAO,45601-402,ETANOL,01/07/2019,"3,49",NaN,R$ / litro,BRANCA


# Experimento Banco de Dados Relacional (SQLite)

In [13]:
from sqlalchemy import create_engine, Column, String, Integer, ForeignKey, DateTime, Float
from sqlalchemy.orm import Session

# Use the default method for abstracting classes to tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [14]:
class Coleta(Base):
    __tablename__ = 'coleta'
    id = Column(Integer, primary_key=True)
    regiao = Column(String(255))
    estado = Column(String(255))
    municipio = Column(String(255))
    revenda = Column(String(255))
    cnpj = Column(String(255))
    rua = Column(String(255))
    numero = Column(String(255))
    complemento = Column(String(255))
    bairro = Column(String(255))
    cep = Column(String(255))
    produto = Column(String(255))
    data = Column(String(255))
    valor_venda = Column(String(255))
    valor_compra = Column(String(255))
    unidade_medida = Column(String(255))
    bandeira = Column(String(255))

In [15]:
# Create DB connection
engine = create_engine("sqlite:///precos.sqlite")
conn = engine.connect()
session = Session(bind=engine)

# Create metadata layer that abstracts our SQL DB
Base.metadata.create_all(engine)

In [16]:
#Create objects
tempo_ini = time.time()
for index, row in df.iterrows():
    session.add(Coleta(
        id = index,
        regiao = row['Regiao - Sigla'],
        estado = row['Estado - Sigla'],
        municipio = row['Municipio'],
        revenda = row['Revenda'],
        cnpj = row['CNPJ da Revenda'],
        rua = row['Nome da Rua'],
        numero = row['Numero Rua'],
        complemento = row['Complemento'],
        bairro = row['Bairro'],
        cep = row['Cep'],
        produto = row['Produto'],
        data = row['Data da Coleta'],
        valor_venda = row['Valor de Venda'],
        valor_compra = row['Valor de Compra'],
        unidade_medida = row['Unidade de Medida'],
        bandeira = row['Bandeira']
    ))
# Commit changes to database
session.commit()
tempo_fim = time.time()
print('### SQL ### ', df.shape[0], 'registros.')
print("Tempo de inserção: {:.2f} segundos".format((tempo_fim - tempo_ini)))

### SQL ###  2563560 registros.
Tempo de inserção: 442.05 segundos


In [17]:
query = """SELECT * FROM coleta AS c WHERE c.regiao = 'SE';"""
tempo_ini = time.time()
result = pd.read_sql_query(query, session.bind)
tempo_fim = time.time()
print('### SQL ### ', result.shape[0], 'registros.')
print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))

### SQL ###  1248130 registros.
Tempo de consulta: 10.18639 segundos


# Experimento Banco de Dados Documentos (MongoDB)

In [18]:
import pymongo

# Establish connection
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Create a database
db = client.precos
db.drop_collection(name_or_collection='coleta')

{'nIndexesWas': 1, 'ns': 'precos.coleta', 'ok': 1.0}

In [19]:
#Create objects
tempo_ini = time.time()
db.coleta.insert_many(df.to_dict('records'))
tempo_fim = time.time()
print('### NoSQL ### ', df.shape[0], 'registros.')
print("Tempo de inserção: {:.2f} segundos".format((tempo_fim - tempo_ini)))

### NoSQL ###  2563560 registros.
Tempo de inserção: 74.26 segundos


In [20]:
find = {'Regiao - Sigla':'SE'}
tempo_ini = time.time()
count = db.coleta.count_documents(find)
tempo_fim = time.time()
print('### NoSQL ### ', count, 'registros.')
print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))

### NoSQL ###  1248130 registros.
Tempo de consulta: 1.95817 segundos
